In [2]:
import os
import deeplabcut
from deeplabcut.utils import xrommtools
import cv2
import numpy as np
from ruamel.yaml import YAML
working_dir=os.getcwd()
experimenter='SD'

In [3]:
# samtools.create_new_project(working_dir, experimenter)
dirs = ["trainingdata", "trials", "XMA_files"]
for dir in dirs:
    try:
        os.mkdir(dir)
    except FileExistsError:
        continue

config = open("project_config.yaml", 'w')

# Create a fake video to pass into the deeplabcut workflow
frame = np.zeros((480, 480, 3), np.uint8)
out = cv2.VideoWriter('dummy.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, (480,480))
out.write(frame)
out.release()

# Create a new project
yaml = YAML()
task = working_dir.split("\\")[len(working_dir.split("\\")) - 1]
path_config_file = deeplabcut.create_new_project(task, experimenter, [working_dir + "\\dummy.avi"], working_dir + "\\", copy_videos=True)
template = f"""
task: {task}
experimenter: {experimenter}
working_dir: {working_dir}
path_config_file: {path_config_file}
dataset_name: 
nframes:
"""

tmp = yaml.load(template)

yaml.dump(tmp, config)
config.close()

try:
    os.rmdir(path_config_file[:path_config_file.find("config")] + "labeled-data\\dummy")
except FileNotFoundError:
    pass

try:
    os.remove(path_config_file[:path_config_file.find("config")] + "\\videos\\dummy.avi")
except FileNotFoundError:
    pass

try:
    os.remove("dummy.avi")
except FileNotFoundError:
    pass

Created "C:\Users\Samuel_DeLap.UMLADCO\Documents\DeLap\XMA2DLC_Pipeline\Code\Code-SD-2022-11-18\videos"
Created "C:\Users\Samuel_DeLap.UMLADCO\Documents\DeLap\XMA2DLC_Pipeline\Code\Code-SD-2022-11-18\labeled-data"
Created "C:\Users\Samuel_DeLap.UMLADCO\Documents\DeLap\XMA2DLC_Pipeline\Code\Code-SD-2022-11-18\training-datasets"
Created "C:\Users\Samuel_DeLap.UMLADCO\Documents\DeLap\XMA2DLC_Pipeline\Code\Code-SD-2022-11-18\dlc-models"
Copying the videos
C:\Users\Samuel_DeLap.UMLADCO\Documents\DeLap\XMA2DLC_Pipeline\Code\Code-SD-2022-11-18\videos\dummy.avi
Generated "C:\Users\Samuel_DeLap.UMLADCO\Documents\DeLap\XMA2DLC_Pipeline\Code\Code-SD-2022-11-18\config.yaml"

A new project with name Code-SD-2022-11-18 is created at C:\Users\Samuel_DeLap.UMLADCO\Documents\DeLap\XMA2DLC_Pipeline\Code and a configurable file (config.yaml) is stored there. Change the parameters in this file to adapt to your project's needs.
 Once you have changed the configuration file, use the function 'extract_frames

In [4]:
# samtools.train_network(working_dir)
config_file = open(working_dir + "\\project_config.yaml", 'r')
yaml = YAML()
project = yaml.load(config_file)

# Establish project vars
path_config_file = project['path_config_file']
data_path = working_dir + "\\trainingdata"
dataset_name = project['dataset_name']
experimenter = str(project['experimenter'])
nframes = project['nframes']

if dataset_name is None:
    raise Exception("Please specify a name for this dataset in the config file")
if nframes is None:
    raise Exception("Please specify the number of frames in the training dataset")

try:
    xrommtools.xma_to_dlc(path_config_file, data_path, dataset_name, experimenter, nframes)
except UnboundLocalError:
    pass
deeplabcut.create_training_dataset(path_config_file)
deeplabcut.train_network(path_config_file)

Extracting camera 1 trial images and 2D points...
Extracting camera 2 trial images and 2D points...
...done.
Training data extracted to projectpath/labeled-data. Now use deeplabcut.create_training_dataset
C:\Users\Samuel_DeLap.UMLADCO\Documents\DeLap\XMA2DLC_Pipeline\Code\Code-SD-2022-11-18\labeled-data\dummy\CollectedData_SD.h5  not found (perhaps not annotated).
Annotation data was not found by splitting video paths (from config['video_sets']). An alternative route is taken...
The following folders were found: ['C:\\Users\\Samuel_DeLap.UMLADCO\\Documents\\DeLap\\XMA2DLC_Pipeline\\Code\\Code-SD-2022-11-18\\labeled-data\\JohnnyAppleseed']


Config:
{'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16],
                [17],
                [18],
                [19],
                [20],
                [21],
                [22],
                [23]],
 'all_joints_names': ['F1',
                      'F2',
                      'LJla',
                      'LJlm',
                      'LJlc',
                      'GHa',
                      'TP',
                      'GHc',
                      'PMX',
                      'AMd',
                      'AMv',
                      'SQr',
                      'SQl',
                      'DMd',
                      'DMv',
                      'BHa',
         

The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!
Selecting single-animal trainer
Batch Size is 1


c:\ProgramData\Anaconda3\envs\DEEPLABCUT\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Loading ImageNet-pretrained resnet_50
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': 'C:\\Users\\Samuel_DeLap.UMLADCO\\Documents\\DeLap\\XMA2DLC_Pipeline\\Code\\Code-SD-2022-11-18\\dlc-models\\iteration-0\\CodeNov18-trainset95shuffle1\\train\\snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'crop_pad': 0, 'scoremap_dir': 'test', 'batch_size': 1, 'dataset_type': 'default', 'deterministic': False, 'mirror': False, 'pairwise_huber_loss': False, 'weigh_only_present_joints': False, 'partaffinityfield_predict': False, 'pairwise_predict': False, 'all_joints': [[0], [1], [2], [3], [4], [5], [6], [7], [8], [

KeyboardInterrupt: 